In [347]:
import numpy as np
import pathlib
import attr
from typing import List, Sequence, Optional

In [4]:
!ls ../../

directional.txt			    LICENSE    src
google-10000-english-no-swears.txt  notebooks  wiki-100k.txt
inappropriate.txt		    README.md  wordlist-eng.txt


In [349]:
class WordList:
    def __init__(
            self,
            wordlist_path: str,
            illegals_paths: Optional[List[str]] = None,
            allowed_paths: Optional[List[str]] = None
    ):
        path = pathlib.Path(wordlist_path)
        self.illegals = self.load_texts(illegals_paths) if illegals_paths else []
        self.allowed = self.load_texts(allowed_paths) if allowed_paths else []

    def load_texts(self, paths: List[str]):
        texts = []
        for pth in paths:
            path = pathlib.Path(pth)
            with path.open() as f:
                texts.extend(f.read().splitlines())
        return [term.strip().upper() for term in texts]

In [350]:
pathlib.Path("../../wordlist-eng.txt").exists()

True

In [352]:
wordlist = WordList(
    "../../wordlist-eng.txt",
    ["../../directional.txt"],
    ["../../wiki-100k.txt"]
)

In [115]:
wordlist

In [16]:
from numpy.random import default_rng
rng = default_rng()

array(['blue', 'bystander', 'blue', 'red', 'blue', 'red', 'blue', 'blue',
       'bystander', 'red', 'red', 'assassin', 'blue', 'bystander',
       'bystander', 'red', 'bystander', 'red', 'blue', 'red', 'blue',
       'bystander', 'bystander', 'red', 'blue'], dtype='<U9')

In [274]:
labels = ["BLUE"] * 9 + ["RED"] * 8 + ["BYSTANDER"] * 7 + ["ASSASSIN"]
unique_labels = np.unique(labels).tolist()

In [218]:
len(labels)

25

In [281]:
def is_superstring_or_substring(word: str, target: str) -> bool:
    return target in word or word in target


class Board:
    def __init__(self, wordlist: WordList) -> None:
        self.wordlist = wordlist
        self.words = rng.choice(wordlist.words, 25, replace=False)
        self.word2index = {word: i for i, word in enumerate(self.words)}
        self.labels = rng.permutation(labels)
        self.reset_game()

    def is_illegal(self, word: str) -> bool:
        word = word.upper()
        return (
            any(is_superstring_or_substring(word, target) for target in self.words) or
            word in self.wordlist.illegals
        )

    def reset_game(self) -> None:
        self.chosen = np.array([False]*25)
        self.which_team_guessing = "BLUE"
        # self.hint_history = []
        # self.state_history = None

    def choose_word(self, word: str) -> str:
        if word.upper() not in self.words:
            raise KeyError(f"Word '{word}' is not on the board.")
        index = self.word2index[word]
        if self.chosen[index]:
            raise ValueError(f"Word '{word}' has already been chosen!")
        self.chosen[index] = True
        return self.labels[index]

    def words_that_are_label(self, label):
        return self.words[self.labels == label]

    @property
    def blue_words(self):
        return words_that_are_label("BLUE")

    @property
    def red_words(self):
        return words_that_are_label("RED")

    @property
    def bystander_words(self):
        return words_that_are_label("BYSTANDER")

    @property
    def assassin_words(self):
        """There is only one assassin in a regular game, but for the sake of generality, here we go!"""
        return words_that_are_label("ASSASSIN")

    def indices_for_label(self, label):
        return np.where(self.labels == label)[0]

    @property
    def blue_indices(self):
        return self.indices_for_label("BLUE")

    @property
    def red_indices(self):
        return self.indices_for_label("RED")

    @property
    def bystander_indices(self):
        return self.indices_for_label("BYSTANDER")

    @property
    def assassin_indices(self):
        """There is only one assassin in a regular game, but for the sake of generality, here we go!"""
        return self.indices_for_label("ASSASSIN")

    def jump_to_random_state(self) -> None:
        """Jump to a valid random state before the end of the game.
        
        There must be 1 assassin, 1-9 blue words, 1-8 red words, and 0-7 bystanders.
        Thus, 0-8 blue, 0-7 red and 1-6 bystander words are chosen.
        """
        self.reset_game()
        num_blue = rng.integers(0, 9)
        num_red = rng.integers(0, 8)
        num_bystanders = rng.integers(0, 7)
        chosen_blue = rng.choice(self.blue_indices, num_blue, replace=False)
        chosen_red = rng.choice(self.red_indices, num_red, replace=False)
        chosen_bystanders = rng.choice(self.bystander_indices, num_bystanders, replace=False)
        chosen_indices = np.concatenate([chosen_blue, chosen_red, chosen_bystanders])
        self.chosen[chosen_indices] = True
        self.which_team_guessing = rng.choice(["BLUE", "RED"])

    def bag_state(self):
        return {label: set(self.words[(self.labels == label) & ~self.chosen]) for label in unique_labels}

In [282]:
rng.choice([0, 1])

0

In [283]:
class CliView:
    def __init__(self, board: Board):
        self.board = board

    def spymaster_words_to_display(self):
        words = []
        # Arguably more readable than the equivalent list comprehension
        for w, l, c in zip(self.board.words, self.board.labels, self.board.chosen):
            w += f"_{l[0]}"
            if c:
                w = w.lower()
            words.append(w) 
        return words

    def operative_words_to_display(self):
        words = []
        # Arguably more readable than the equivalent list comprehension
        for w, l, c in zip(self.board.words, self.board.labels, self.board.chosen):
            if c:
                w += f"_{l[0]}"
                w = w.lower()
            words.append(w)
        return words

    def generic_view(self, words_to_display):
        words = words_to_display()
        print(np.array(words).reshape(5,5))
        print(f"It is {self.board.which_team_guessing}'s turn.")

    def spymaster_view(self):
        self.generic_view(self.spymaster_words_to_display)

    def operative_view(self):
        self.generic_view(self.operative_words_to_display)

In [284]:
np.concatenate([np.arange(3), np.arange(2)])

array([0, 1, 2, 0, 1])

In [285]:
board = Board(wordlist)
view = CliView(board)

In [286]:
view.spymaster_view()

[['LINE_B' 'CONTRACT_A' 'PANTS_B' 'SPELL_R' 'PASTE_R']
 ['BUFFALO_B' 'HORSE_B' 'SPINE_R' 'BERRY_B' 'INDIA_B']
 ['BOLT_B' 'STOCK_B' 'GRASS_B' 'SHOT_B' 'MOON_B']
 ['PIPE_B' 'MILLIONAIRE_R' 'SPY_B' 'CAST_R' 'AMAZON_B']
 ['FORCE_R' 'YARD_R' 'VET_R' 'PRESS_B' 'EUROPE_B']]
It is BLUE's turn.


In [291]:
board.bag_state()

{'ASSASSIN': {'CONTRACT'},
 'BLUE': {'AMAZON', 'BUFFALO', 'EUROPE', 'LINE', 'PIPE', 'STOCK'},
 'BYSTANDER': {'BERRY', 'GRASS', 'INDIA', 'PANTS', 'SHOT'},
 'RED': {'VET'}}

In [288]:
board.blue_indices

array([ 0,  5, 10, 11, 15, 17, 19, 23, 24])

In [289]:
board.jump_to_random_state()

In [290]:
view.operative_view()

[['LINE' 'CONTRACT' 'PANTS' 'spell_r' 'paste_r']
 ['BUFFALO' 'horse_b' 'spine_r' 'BERRY' 'INDIA']
 ['bolt_b' 'STOCK' 'GRASS' 'SHOT' 'moon_b']
 ['PIPE' 'millionaire_r' 'spy_b' 'cast_r' 'AMAZON']
 ['force_r' 'yard_r' 'VET' 'press_b' 'EUROPE']]
It is BLUE's turn.


In [300]:
@attr.s(frozen=True, auto_attribs=True)
class Hint:
    word: str
    count: str

In [303]:
class GloveGuesser:
    def __init__(self, glove_path):
        self.glove_path = glove_path

    def guess(self, board: Board, hint: Hint) -> Sequence[str]:
        pass

In [310]:
!ls ../../../codenames/dataset/glove

ls: cannot access '../../../codenames/dataset/glove': No such file or directory


In [316]:
glove_path = pathlib.Path("../../../codenames/dataset/glove.6B.300d.npy")


In [319]:
with glove_path.open("rb") as f:
    glove_vectors = np.load(f)

In [339]:
class Glove:
    def __init__(self, glove_vector_path, glove_tokens_path):
        gv_path = pathlib.Path(glove_vector_path)
        gt_path = pathlib.Path(glove_tokens_path)
        assert gv_path.exists()
        assert gt_path.exists()
        with gv_path.open("rb") as f:
            self.vectors = np.load(gv_path)
        with gt_path.open() as f:
            self.tokens = f.read().splitlines()
            self.tokens = [x.strip().upper() for x in self.tokens]
        self.token2id = {t: i for i, t in enumerate(self.tokens)}
    def tokenize(self, phrase):
        """Simple one-word tokenization. Ignores punctuation."""
        phrase = phrase.strip().upper()
        return [self.token2id[x] if x in self.token2id else None for x in phrase.split()]

In [340]:
glove = Glove("../../../codenames/dataset/glove.6B.300d.npy", "../../../codenames/dataset/words")

In [346]:
glove.tokenize(" ".join(wordlist.words))

[637,
 1967,
 325,
 8334,
 12038,
 8427,
 9214,
 453,
 5239,
 12451,
 3292,
 2647,
 12339,
 603,
 26900,
 137,
 1083,
 775,
 231,
 2069,
 14924,
 4925,
 5747,
 1497,
 3045,
 960,
 3827,
 942,
 2913,
 5077,
 2499,
 11012,
 9307,
 480,
 1963,
 534,
 11869,
 1211,
 1846,
 4707,
 10002,
 6676,
 7331,
 1930,
 1641,
 9248,
 8998,
 4794,
 11035,
 41652,
 6910,
 14105,
 774,
 3539,
 351,
 569,
 1904,
 20786,
 5391,
 1784,
 5450,
 2114,
 46995,
 313,
 3845,
 511,
 1090,
 2375,
 5778,
 17489,
 132,
 6242,
 512,
 4012,
 8525,
 23755,
 449,
 2280,
 1866,
 4249,
 3990,
 3031,
 8475,
 953,
 3387,
 5139,
 5223,
 202,
 1333,
 10216,
 2005,
 2162,
 1007,
 3120,
 4124,
 2312,
 2261,
 1257,
 122,
 336,
 6744,
 1714,
 5188,
 14621,
 13308,
 1289,
 2082,
 2926,
 1737,
 7394,
 4635,
 8416,
 1560,
 7774,
 15389,
 5838,
 1598,
 1847,
 2100,
 563,
 525,
 2090,
 621,
 1791,
 807,
 3267,
 6043,
 307263,
 3510,
 1265,
 2854,
 319,
 484,
 2120,
 16677,
 2361,
 2149,
 352,
 2061,
 11532,
 387,
 186,
 851,
 9780,
 5

In [320]:
glove_vectors

array([[ 0.04656  ,  0.21318  , -0.0074364, ...,  0.0090611, -0.20989  ,
         0.053913 ],
       [-0.25539  , -0.25723  ,  0.13169  , ..., -0.2329   , -0.12226  ,
         0.35499  ],
       [-0.12559  ,  0.01363  ,  0.10306  , ..., -0.34224  , -0.022394 ,
         0.13684  ],
       ...,
       [ 0.075713 , -0.040502 ,  0.18345  , ...,  0.21838  ,  0.30967  ,
         0.43761  ],
       [ 0.81451  , -0.36221  ,  0.31186  , ...,  0.075486 ,  0.28408  ,
        -0.17559  ],
       [ 0.429191 , -0.296897 ,  0.15011  , ...,  0.28975  ,  0.32618  ,
        -0.0590532]], dtype=float32)